# Motion Coherence Experiment Analysis Flow

## Glossary of Variable Names:
* moz/MOz = Mert Özkan
* mc: motion coherence
* f: file
* dat: data
* l: line
* idx: index
* ptr: pointer
* trl: trial
* no: number (rank)
* dxn: direction
* coh: coherence
* ok: correct
* is: prefix such as isOK = is it correct?
* rt: reaction time
* t: time
* st: string
* i: integer
* stc: sentence
* w: word
* inv: inventory
* cond: condition
* sq: order/sequence
* sxn: session
* rej: rejected
* sub: subject/participant
* w: with
* wh: *prefix* which/what
* prev: previous

## Synchronising Psychophysical and EEG Trials

* Discard the trials in the psychophysical data which were removed in the artefact rejection.
    * There are two-fold steps of artefact rejection implemented in the Brainvision Analyzer. So, 
* Remove invalid trials and keep them logged to discard from EEG data:
    * 1s > RT(from onset) > .1
    * Invalid key press
* Mark correct and wrong trials

### Import Data from Log Files

In [13]:
def import_psyphys_mc(f_name,path):
    '''
    Feb 22, 2019
    Mert Ozkan
    Dartmouth College
    Motion Coherence
    
    Imports datasets from log files for the Motion Coherence experiment.
    Usage: trl_no, dxn, coh, isOK, key, rt = import_psyphys_mc(f_name,path)
    '''
    
    import numpy as np
    
    dat_path = '/'.join([path,f_name])
    f = open(dat_path,'r')
    
    # Data log starts 2 lines after the data pointer
    dat_ptr = '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%'
    read_l = f.readlines()
    for idx in range(len(read_l)):
        if dat_ptr in read_l[idx]:
            dat_start_idx = idx + 2
            break
    trl_no = []
    dxn = []
    coh = []
    isOK = []
    key = []
    rt = [] 
    for trl in read_l[dat_start_idx:]:
        trl_dat = trl.split()
        trl_no.append(int(trl_dat[0]))
        dxn.append(int(trl_dat[1]))
        coh.append(float(trl_dat[2]))
        isOK.append(int(trl_dat[3]))
        key.append(trl_dat[4])
        rt.append(float(trl_dat[5]))
    
    trl_no = np.array(trl_no)
    dxn = np.array(dxn)
    coh = np.array(coh)
    isOK = np.array(isOK)
    key = np.array(key)
    rt = np.array(rt)
    return trl_no, dxn, coh, isOK, key, rt

### Import Subject Data inventory

In [5]:
def import_subjectdatainfo_mc(path):
    '''
    Feb 22, 2019
    Mert Ozkan
    Dartmouth College
    Motion Coherence
    
    Import subject data inventory from data_inventory_mc.txt
    Usage: sub_no, sub_name, sq, bhv_ptr, eeg_ptr, trl_rej_1, trl_rej_2 = import_subjectdatainfo_mc(path)
    '''
    
    import numpy as np
    
    f = open(path,'r')
    read_l = f.readlines()
    
    sub_no = []
    sub_name = []
    sq = []
    bhv_ptr = []
    eeg_ptr = []
    trl_rej_1 = []
    trl_rej_2 = []
    for idx in range(len(read_l)):
        if '*' in read_l[idx]:
            inv_st = read_l[idx+1].split('; ')
            print(inv_st)
            sub_no.append(int(inv_st[0]))
            sub_name.append(inv_st[1])
            sq.append(
                    np.array(inv_st[2].split(', ')).astype(int))
            bhv_ptr.append(inv_st[3])
            eeg_ptr.append(inv_st[4])
            trl_rej_1.append(
                    np.array(inv_st[5].split(', ')).astype(int))
            if inv_st[6][-1:] == '\n':
                inv_st[6] = inv_st[6][:-1]
            if inv_st[6] != '':
                trl_rej_2.append(
                    np.array(inv_st[6].split(', ')).astype(int))
            else:
                trl_rej_2.append(
                    np.array([]).astype(int))
    return sub_no, sub_name, sq, bhv_ptr, eeg_ptr, trl_rej_1, trl_rej_2
    
path = '/Users/mertozkan/Documents/Psyc161FinalProject/MotionCoherence/MC/data_inventory_mc.txt'
sub_no, sub_name, sq, bhv_ptr, eeg_ptr, trl_rej_1, trl_rej_2 = import_subjectdatainfo_mc(path)

['1', 'MOz', '2, 3, 1', 'mertozkan', 'MO', '39, 73, 90, 123, 130, 187, 216, 225, 227, 365, 380, 382, 468, 470, 514, 593, 598, 634, 637', '53, 169, 276, 352, 353, 354, 406, 407, 424, 583, 634, 649, 666, 667\n']
['2', 'SA', '3, 1, 2', 'sibelakyuz', 'SA', '159, 213, 227, 228, 244, 245, 294, 356, 370, 376, 449, 456, 477, 487, 489, 495, 497, 572, 580, 595, 607, 609, 610, 642, 657, 665, 668, 669, 670, 671, 677, 681, 685, 694, 695, 697, 698, 699, 700, 710, 711, 725', '190, 198, 376, 425, 620\n']
['3', 'ZP', '2, 1, 3', 'zp', 'ZP', '1, 6, 26, 49, 66, 129, 179, 180, 181, 252, 266, 299, 334, 394, 416, 422, 479, 539, 559, 568, 596, 652, 653, 655, 668, 676, 684, 702, 713, 714, 715, 721, 733, 735, 738, 770, 778, 779, 791, 792', '135, 393\n']
['4', 'AYO', '1, 2, 3', 'ayo', 'AYO', '91, 152, 153, 154, 156, 158, 168, 252, 300, 332, 352, 354, 355, 356, 357, 439, 478, 576, 599, 617, 674, 675, 707', '65, 82, 435, 458, 544\n']
['5', 'BMU', '3, 1, 2', 'bmu', 'BMU', '70, 71, 97, 149, 154, 162, 243, 268, 269, 

In [7]:
def findfilenameindirectory_mc(path, namecontains, end_w):
    import os
    contents = os.listdir(path)
    for i in contents:
        if namecontains in i and end_w in i:
            f_name = i
            break
    return f_name

path = '/Users/mertozkan/Documents/Psyc161FinalProject/MotionCoherence/data'
findfilenameindirectory_mc(path,'ENC','translational.log')

'ENC_9-20_17-47_translational.log'

In [25]:
def combineconditions_mc():#(sub_name,sq, bhv_ptr,trl_rej_1,trl_rej_2):
    import numpy as np
    import os
    
    sq = [2, 1, 0]
    path = '/Users/mertozkan/Documents/Psyc161FinalProject/MotionCoherence/data'
    whSub = 'ENC'
    
    conds = np.array(['translational.log', 'rotational.log', 'radial.log'])
    conds = conds[sq]
    
    prev_sxn_trl_no = 0
    prev_trl_no = np.array([])
    prev_dxn = np.array([])
    prev_coh = np.array([])
    prev_isOK = np.array([])
    prev_key = np.array([])
    prev_rt = np.array([])
    
    for cond in conds:
        f_name = findfilenameindirectory_mc(path, whSub, cond)
        trl_no, dxn, coh, isOK, key, rt = import_psyphys_mc(f_name,path)
        
        trl_no += prev_sxn_trl_no
        prev_sxn_trl_no = trl_no[-1]
        
        prev_trl_no = np.concatenate((prev_trl_no,trl_no), axis=None)
        prev_dxn = np.concatenate((prev_dxn,dxn), axis=None)
        prev_coh = np.concatenate((prev_coh,coh), axis=None)
        prev_isOK = np.concatenate((prev_isOK,isOK), axis=None)
        prev_key = np.concatenate((prev_key,key), axis=None)
        prev_rt = np.concatenate((prev_rt,rt), axis=None)
        
    return prev_trl_no, prev_dxn, prev_coh, prev_isOK, prev_key, prev_rt
combineconditions_mc()

(array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
         12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
         23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
         34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
         45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
         56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
         67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
         78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
         89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
        100., 101., 102., 103., 104., 105., 106., 107., 108., 109., 110.,
        111., 112., 113., 114., 115., 116., 117., 118., 119., 120., 121.,
        122., 123., 124., 125., 126., 127., 128., 129., 130., 131., 132.,
        133., 134., 135., 136., 137., 138., 139., 140., 141., 142., 143.,
        144., 145., 146., 147., 148., 